In [1]:
import sys
sys.path.insert(0, '/usr/src/mytonctrl')

In [2]:
import requests as R
import mytonctrl as M


from datetime import datetime

# TPS, validators, shards

In [15]:
ton = M.MyTonCore()


data = {}
data['tps'] = ton.GetStatistics('tpsAvg')[0]

# block rate
data['block_rate'] = {
    'masterchain': 3.0,
    'basechain': 2.0,
}

data['shardchains'] = ton.GetShardsNumber()

# validators
validators = ton.GetValidatorsList()
validators_total = len(validators)
validators_online = len(list(filter(lambda x: x['online'], validators)))
data['validators'] = {
    'online': validators_online,
    'total': validators_total
}

data

{'tps': 1.3,
 'block_rate': {'masterchain': 3.0, 'basechain': 2.0},
 'shardchains': 1,
 'validators': {'online': 124, 'total': 139}}

# dht servers

In [186]:
import socket
import struct
import json
from loguru import logger
from tcp_latency import measure_latency


def ip2int(addr):
    return struct.unpack("!i", socket.inet_aton(addr))[0]


def int2ip(addr):
    return socket.inet_ntoa(struct.pack("!i", addr))

In [232]:
# Считать список серверов
filePath = ton.liteClient.configPath
file = open(filePath, 'rt')
text = file.read()
file.close()
data = json.loads(text)
dhts = data['dht']

dht_servers = []
for dht in dhts['static_nodes']['nodes']:
    addr_dict = dht['addr_list']['addrs'][0]
    if len(dht['addr_list']['addrs']) > 1:
        logger.warning("WARNING, more than 1 address!")
    
    ip = int2ip(addr_dict['ip'])
    port = addr_dict['port']
    addr = f"{ip}:{port}"
    if ip == "127.0.0.1":
        continue
        
    # ping
    runs = 5
    resp = measure_latency(addr, runs=runs, wait=0, timeout=5)
    resp = [x for x in resp if x is not None]
    if len(resp) > 0:
        resp_time = sum(resp) / len(resp)
        available = True
    else:
        resp_time = None
        available = '?'
    
    loc = {'addr': addr, 'available': '?', 'response_time': resp_time}
    dht_servers.append(loc)
dht_servers

[{'addr': '178.18.243.125:15888', 'available': '?', 'response_time': None},
 {'addr': '178.18.243.132:15888', 'available': '?', 'response_time': None},
 {'addr': '178.18.243.136:15888', 'available': '?', 'response_time': None},
 {'addr': '185.86.77.139:26907', 'available': '?', 'response_time': None},
 {'addr': '92.255.207.93:29503', 'available': '?', 'response_time': None},
 {'addr': '139.162.201.65:14395', 'available': '?', 'response_time': None},
 {'addr': '172.104.59.125:14432', 'available': '?', 'response_time': None},
 {'addr': '172.105.29.108:14583', 'available': '?', 'response_time': None},
 {'addr': '65.21.7.173:15813', 'available': '?', 'response_time': None},
 {'addr': '162.55.50.96:12821', 'available': '?', 'response_time': None},
 {'addr': '23.88.40.211:15895', 'available': '?', 'response_time': None},
 {'addr': '164.90.201.195:18513', 'available': '?', 'response_time': None},
 {'addr': '161.35.49.42:11369', 'available': '?', 'response_time': None},
 {'addr': '142.93.219.1

# check liteservers

In [211]:
# Считать список серверов
filePath = ton.liteClient.configPath
file = open(filePath, 'rt')
text = file.read()
file.close()
data = json.loads(text)

# Пройтись по серверам
result = list()
liteservers = data.get("liteservers")
for index, server in enumerate(liteservers):
    ip = int2ip(server['ip'])
    port = server['port']
    addr = f"{ip}:{port}"
    
    # working
    working = False
    try:
        ton.liteClient.Run("last", index=index)
        working = True
    except: pass
    
    # ping
    if working:
        runs = 10
        resp_time = measure_latency(ip, port=port, runs=runs, wait=0)
        ping = sum(resp_time) / runs
    else:
        ping = None
    
    # status
    try:
        resp = ton.liteClient.Run("allshards", index=index)
        print(resp)
    except: pass
    
    loc = {
        'addr': addr,
        'working': working,
        'response_time': ping,
        'syncronized': 'unknown',
    }
    print(loc)
    break
    result.append(loc)
#end for

# Записать данные в базу
result

using liteserver 0 with addr [67.207.74.182:4924]
zerostate set to -1:17A3A92992AABEA785A7A090985A265CD31F323D849DA51239737E321FB05569:5E994FCF4D425C0A6CE6A792594B7173205F740A39CD56F537DEFD28B48A0F6E
latest masterchain block known to server is (-1,8000000000000000,16087396):6CC4D9B95E2F28950735773C81B8E9FEAD5A1B1C6595425EC4925A17EADAE81C:9E77FAD7A1DAB400485C4413F331F6B5EC446FF2C1C62EF713E215D5EFEDCD72 created at 1636927125 (7 seconds ago)
BLK#1 = (-1,8000000000000000,16087396):6CC4D9B95E2F28950735773C81B8E9FEAD5A1B1C6595425EC4925A17EADAE81C:9E77FAD7A1DAB400485C4413F331F6B5EC446FF2C1C62EF713E215D5EFEDCD72
BLK#2 = (-1,8000000000000000,0):17A3A92992AABEA785A7A090985A265CD31F323D849DA51239737E321FB05569:5E994FCF4D425C0A6CE6A792594B7173205F740A39CD56F537DEFD28B48A0F6E
shard configuration is (
  (hme_root
    root:(hm_edge
      label:(hml_same v:0 n:32)
      node:(hmn_leaf
        value:^(bt_leaf
          leaf:(shard_descr_new seq_no:20933481 reg_mc_seqno:16087396 start_lt:22999808000000 

[]

# Services uptime

In [230]:
services_list = [
    {
        'name': 'ton.sh',
        'api': 'https://toncenter.com/api/v2/getAddressInformation?address=0QCyt4ltzak71h6XkyK4ePfZCzJQDSVUNuvZ3VE7hP_Q-GTE'

    }
]

# Governance

In [229]:
config17 = ton.GetConfig17()
config15 = ton.GetConfig15()
config34 = ton.GetConfig34()
config36 = ton.GetConfig36()

fullElectorAddr = ton.GetFullElectorAddr()


# get times
startWorkTime = ton.GetActiveElectionId(fullElectorAddr)
election_status = "open"
if startWorkTime == 0:
    startWorkTime = config36.get("startWorkTime")
    if startWorkTime is None:
        startWorkTime = config34.get("startWorkTime")
    election_status = "closed" 

electionsStartBefore = config15["electionsStartBefore"]
electionsEndBefore = config15["electionsEndBefore"]
validatorsElectedFor = config15["validatorsElectedFor"]

# compute election times
startElection = startWorkTime - electionsStartBefore
endElection = startWorkTime - electionsEndBefore
startNextElection = startElection + validatorsElectedFor

startElection = datetime.fromtimestamp(startElection)
endElection = datetime.fromtimestamp(endElection)
startNextElection = datetime.fromtimestamp(startNextElection)

elections = {
    'status': election_status,
    'start': startElection,
    'end': endElection,
    'start_next': startNextElection
}
elections

[debug]   14.11.2021, 22:14:25.374 (UTC)  <MainThread>  start GetConfig function (17)
[debug]   14.11.2021, 22:14:25.388 (UTC)  <MainThread>  start GetConfig function (15)
[debug]   14.11.2021, 22:14:25.401 (UTC)  <MainThread>  start GetConfig36 function
[debug]   14.11.2021, 22:14:25.417 (UTC)  <MainThread>  start GetFullElectorAddr function
[debug]   14.11.2021, 22:14:25.430 (UTC)  <MainThread>  start GetActiveElectionId function


{'status': 'closed',
 'start': datetime.datetime(2021, 11, 14, 14, 0, 39),
 'end': datetime.datetime(2021, 11, 14, 20, 50, 15),
 'start_next': datetime.datetime(2021, 11, 15, 8, 12, 55)}

In [228]:
offers = ton.GetOffersNumber()
offers_list = ton.GetOffers()
offers['offers_list'] = offers_list

offers

[debug]   14.11.2021, 22:14:22.412 (UTC)  <MainThread>  start GetOffersNumber function
[debug]   14.11.2021, 22:14:22.413 (UTC)  <MainThread>  start GetOffers function
[debug]   14.11.2021, 22:14:22.413 (UTC)  <MainThread>  start GetFullConfigAddr function
[debug]   14.11.2021, 22:14:22.463 (UTC)  <MainThread>  start GetConfig34 function
[debug]   14.11.2021, 22:14:22.479 (UTC)  <MainThread>  start GetOffers function


{'all': 0, 'new': 0, 'offers_list': []}

In [222]:
complaints = ton.GetComplaintsNumber()
complaints_list = ton.GetComplaints()
complaints['complaints_list'] = complaints_list

complaints

[debug]   14.11.2021, 22:08:26.077 (UTC)  <MainThread>  start GetComplaintsNumber function
[debug]   14.11.2021, 22:08:26.077 (UTC)  <MainThread>  start GetComplaints function
[debug]   14.11.2021, 22:08:26.077 (UTC)  <MainThread>  start GetFullElectorAddr function
[debug]   14.11.2021, 22:08:26.099 (UTC)  <MainThread>  start GetConfig32 function
[debug]   14.11.2021, 22:08:26.138 (UTC)  <MainThread>  start GetComplaints function


{'all': 0, 'new': 0, 'complaints_list': {}}

In [115]:
governance = {
    'elections': elections,
    'offers': offers,
    'complaints': complaints
}
governance

{'elections': {'status': 'open',
  'start': datetime.datetime(2021, 11, 14, 14, 0, 39),
  'end': datetime.datetime(2021, 11, 14, 20, 50, 15),
  'start_next': datetime.datetime(2021, 11, 15, 8, 12, 55)},
 'offers': {'all': 0, 'new': 0, 'offers_list': []},
 'complaints': {'all': 0, 'new': 0, 'complaints_list': {}}}

# Bridge check

In [165]:
from bs4 import BeautifulSoup


bridge_list = [{'name': 'ton-eth','url': 'https://ton.org/bridge/', 
                'address': 'Ef_dJMSh8riPi3BTUTtcxsWjG8RLKnLctNjAM4rw8NN-xWdr'},
               {'name': 'ton-bsc', 'url': 'https://ton.org/bridge/bsc', 
                'address': 'Ef9NXAIQs12t2qIZ-sRZ26D977H65Ol6DQeXc5_gUNaUys5r'},
               {'name': 'ton-other', 'url': 'https://ton.org/bridge/exmo', 
                'address': ''}]

bridges = []
for bridge_dict in bridge_list:
    resp = R.get(bridge_dict['url'])
    
    # web page status
    web_page_status = 'not available'
    if resp.status_code == 200:
        page = BeautifulSoup(resp.content, features="html.parser")
        button = page.find_all(attrs={'id': ['transferButton']})
        if len(button) > 0:
            web_page_status = 'available'
            
    # api status
    smart_contract_state = None
    if bridge_dict['address']:
        cmd = 'runmethod {address} get_bridge_data'.format(**bridge_dict)
        resp = ton.liteClient.Run(cmd)
        for row in resp.split('\n'):
            if 'result:' in row and '[' in row:
                row = row[row.find('[')+1:row.find(']')].strip()
                row = row.split()
                if len(row) > 0:
                    smart_contract_state = int(row[0])
    bridges.append({
        'name': bridge_dict['name'],
        'url': bridge_dict['url'],
        'web_page_status': web_page_status,
        'smart_contract_state': smart_contract_state
    })
bridges

[{'name': 'ton-eth',
  'url': 'https://ton.org/bridge/',
  'web_page_status': 'available',
  'smart_contract_state': 0},
 {'name': 'ton-bsc',
  'url': 'https://ton.org/bridge/bsc',
  'web_page_status': 'available',
  'smart_contract_state': 0},
 {'name': 'ton-other',
  'url': 'https://ton.org/bridge/exmo',
  'web_page_status': 'not available',
  'smart_contract_state': None}]

# OLD

In [ ]:
lite = M.LiteClient()

In [ ]:
ton.GetConfig12()

In [ ]:
ton.GetConfig15()

In [ ]:
config36 = ton.GetConfig36()
config36

In [ ]:
shardsNumber = ton.GetShardsNumber()
print('shards_number:', shardsNumber)

In [ ]:
online_validators = ton.GetOnlineValidators()
validator_efficiency = ton.GetValidatorEfficiency()

In [ ]:
config15 = ton.GetConfig15()
config17 = ton.GetConfig17()

In [ ]:
statistics = ton.GetSettings("statistics")
statistics

In [ ]:
rootWorkchainEnabledTime_int = ton.GetRootWorkchainEnabledTime()
rootWorkchainEnabledTime_int

In [ ]:
validators_info = ton.GetValidatorsLoad()
validators_info[0]

In [ ]:
compliants_info = ton.GetComplaints()
compliants_info

In [ ]:
ton.GetAdnlAddr()

In [ ]:
statistics = ton.GetSettings("statistics")
tpsAvg = ton.GetStatistics("tpsAvg", statistics)
netLoadAvg = ton.GetStatistics("netLoadAvg", statistics)
disksLoadAvg = ton.GetStatistics("disksLoadAvg", statistics)
disksLoadPercentAvg = ton.GetStatistics("disksLoadPercentAvg", statistics)

statistics

In [ ]:
def PrintStatus(args):
    opt = None
    if len(args) == 1:
        opt = args[0]
    rootWorkchainEnabledTime_int = ton.GetRootWorkchainEnabledTime()
    config34 = ton.GetConfig34()
    config36 = ton.GetConfig36()
    totalValidators = config34["totalValidators"]
    onlineValidators = None
    validatorEfficiency = None
    if opt != "fast":
        onlineValidators = ton.GetOnlineValidators()
        validatorEfficiency = ton.GetValidatorEfficiency()
    if onlineValidators:
        onlineValidators = len(onlineValidators)
    oldStartWorkTime = config36.get("startWorkTime")
    if oldStartWorkTime is None:
        oldStartWorkTime = config34.get("startWorkTime")
    shardsNumber = ton.GetShardsNumber()
    validatorStatus = ton.GetValidatorStatus()
    config15 = ton.GetConfig15()
    config17 = ton.GetConfig17()
    fullConfigAddr = ton.GetFullConfigAddr()
    fullElectorAddr = ton.GetFullElectorAddr()
    startWorkTime = ton.GetActiveElectionId(fullElectorAddr)
    validatorIndex = ton.GetValidatorIndex()
    validatorWallet = ton.GetLocalWallet(ton.validatorWalletName)
    dbSize = ton.GetDbSize()
    offersNumber = ton.GetOffersNumber()
    complaintsNumber = ton.GetComplaintsNumber()
    statistics = ton.GetSettings("statistics")
    tpsAvg = ton.GetStatistics("tpsAvg", statistics)
    netLoadAvg = ton.GetStatistics("netLoadAvg", statistics)
    disksLoadAvg = ton.GetStatistics("disksLoadAvg", statistics)
    disksLoadPercentAvg = ton.GetStatistics("disksLoadPercentAvg", statistics)
    if validatorWallet is not None:
        validatorAccount = ton.GetAccount(validatorWallet.addr)
    else:
        validatorAccount = None
    PrintTonStatus(startWorkTime, totalValidators, onlineValidators, shardsNumber, offersNumber, complaintsNumber, tpsAvg)
    PrintLocalStatus(validatorIndex, validatorEfficiency, validatorWallet, validatorAccount, validatorStatus, dbSize, netLoadAvg, disksLoadAvg, disksLoadPercentAvg)
    PrintTonConfig(fullConfigAddr, fullElectorAddr, config15, config17)
    PrintTimes(rootWorkchainEnabledTime_int, startWorkTime, oldStartWorkTime, config15)